# First implementation of our model

Have a look at the model [here](https://github.com/commons-research/common_dws_public_storage/blob/main/docs/anticipated_lotus/model/Metabolites.pdf). 

* Simulate $P_m$ and $Q_s$ from a Poisson distribution
* Simulate $\sigma_{tf}$ using a Wishart distribution

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import pandas as pd
import scipy.special

### Choose the number of molecules and species

In [2]:
T = ['m', 's']
n_t = [1000, 100]

In [3]:
assert len(T)==len(n_t)

### Create some $\mu$, $\alpha$ and $\beta$ for each molecule and and each species

In [4]:
def simulate_from_prior(T, n_t):
    mu = {T[i]: np.random.normal(loc=0, scale=1, size=n_t[i]) for i in range(len(T))}
    
    alpha = {T[i]: np.random.exponential(scale=2, size=1) for i in range(len(T))}
    
    beta = {T[i]: np.random.exponential(scale=3, size=n_t[i]) for i in range(len(T))}
    
    sigma = {T[i]: stats.wishart.rvs(df=n_t[i],
                                     scale=1/n_t[i]*np.eye(n_t[i]),
                                     size=1) for i in range(len(T))}
    
    return mu, alpha, beta, sigma

In [5]:
def simulate_data(T, n_t):
    return np.random.binomial(n=1, p=0.1, size=[i for i in n_t])

In [6]:
μ, α, β, σ= simulate_from_prior(T, n_t)

In [7]:
x = simulate_data(T, n_t)

In [8]:
γ = np.random.exponential(scale=1, size=1)

In [9]:
δ = np.random.exponential(scale=0.1, size=1)

Change to 1+ Poisson --> We shouldn't change to 1 + Poisson. I fact, the Lotus database will very likely be smaller than the true data **x**. This is why some molecules or some species might not have any research papers yet. 

In [10]:
#P_m = np.random.poisson(0.5, size=n_t[0])

In [11]:
L = np.random.poisson(0.01, size=[i for i in n_t])

In [12]:
Q_s = L.sum(axis=0)

In [13]:
P_m = L.sum(axis=1)

In [14]:
#Q_s = 1 + np.random.poisson(0.5, size=n_t[1])

In [15]:
def research_effort(γ, δ, P_m, Q_s):
    return 1 - np.exp(-γ * P_m[:, None] - δ * Q_s[None, :])

In [16]:
#R = research_effort(γ, δ, P_m, Q_s)

In [17]:
#look at the condition in the model to calculate the proba of L_sm
def compute_prob_of_L(L, x, γ, δ, P_m, Q_s):
    prob_L = np.zeros(L.shape)
    R_ms = 1 - np.exp(-γ * P_m[:, None] - δ * Q_s[None, :])
    
    condition_2 = (x == 0) & (L == 0)
    condition_3 = np.where((x == 1) & (L >= 1))
    condition_4 = np.where((x == 1) & (L == 0))
    
    prob_L[condition_2] = 1
    prob_L[condition_3] = R_ms[condition_3]
    prob_L[condition_4] = 1 - R_ms[condition_4]
    
    return prob_L

In [18]:
prob_L = compute_prob_of_L(L, x, γ, δ, P_m, Q_s)

In [19]:
#df = pd.DataFrame(np.array([L.flatten(), prob_L.flatten()]))

## Calculate proba of **$x$**

In [20]:
def prob_X(mu):
    #extract each array of the mu dictionary
    arrays = list(mu.values())
    
    #create a meash of each combination of each value
    mesh = np.meshgrid(*arrays, indexing='ij')

    sum_combinations = np.sum(mesh, axis=0)
    return scipy.special.expit(sum_combinations)

In [21]:
prob_X(μ).shape

(1000, 100)

In [22]:
def prob_sigma(alpha, sigma):
    multiplied_matrices = {key: np.ndarray.flatten(alpha[key] * sigma[key]).astype('float16') for key in alpha.keys()}
    
    matrices = list(multiplied_matrices.values())
    
    mesh = np.meshgrid(*matrices, indexing='ij')
    return np.sum(mesh, axis=0)

In [ ]:
pd.DataFrame(prob_sigma(α, σ))

In [ ]:
α

In [ ]:
σ['m'][1,0]

In [ ]:
σ['s'][0,0]